# Dynamic plot of ACE IMF data with Bokeh

This example needs to be run localy on your machine with `jupyter_bokeh` installed

In [1]:
import speasy as spz
from speasy.inventory.data_tree import amda as amdatree
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display

spz.config.proxy_enabled.set("true")
spz.config.proxy_url.set("http://sciqlop.lpp.polytechnique.fr/cache")

from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10_10
from bokeh.layouts import gridplot
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [17]:
spz.inventory.flat_inventories.amda.parameters['imf_mag']

<ParameterIndex: |b|>

In [18]:
widgets.Combobox(
    # value='John',
    placeholder='Choose A product',
    options=[key for key in spz.inventory.flat_inventories.amda.parameters.keys()],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)

Combobox(value='', description='Combobox:', ensure_option=True, options=('imf_mag', 'imf', 'imf_gsm', 'imf_rea…

In [3]:
def get_product_index(key):
    return spz.inventory.flat_inventories.amda.parameters[key]

start=datetime(2019, 1,10)
end=datetime(2019, 1,11)

product = 'imf_gsm'


p=figure(x_axis_type="datetime", tools="pan,wheel_zoom,xwheel_pan,box_zoom,reset")
df=spz.amda.get_data(get_product_index(product), start, end).to_dataframe(datetime_index=True)
source = ColumnDataSource(df)

def update_product(new_product):
    global product, p
    product = new_product
    

def app(doc):
    
    
    color = Category10_10.__iter__()
    
    for col in df.columns:
        p.line(x='index',y=col,source=source,color=next(color))

    def x_range_changed(attr, old, new):
        global start
        global end
        if attr == 'start':
            start = datetime.utcfromtimestamp(new/1000.)
        elif attr == 'end':
            end = datetime.utcfromtimestamp(new/1000.)
            source.data = spz.amda.get_data(get_product_index(product), start, end).to_dataframe(datetime_index=True)
            #source.update()

    p.x_range.on_change('start', x_range_changed)
    p.x_range.on_change('end', x_range_changed)

    doc.add_root(gridplot([[p]], sizing_mode='stretch_both'))


products=widgets.Combobox(
    # value='John',
    placeholder='Choose A product',
    options=[key for key in spz.inventory.flat_inventories.amda.parameters.keys()],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)

#products.observe(lambda v: update_product(v['new']), names='value')

display(products)
show(app)


Combobox(value='', description='Combobox:', ensure_option=True, options=('imf_mag', 'imf', 'imf_gsm', 'imf_rea…